### 절대 모멘텀
- 파생변수 'STD-YM' 생성 -> index에서 년-월 추출하여 대입
- 'STD-YM' 별 마지막날의 데이터를 month_last_df 저장
- 전월의 수정주가 값을 가진 파생변수를 생성 
- 전년도의 수정주가 값을 가진 파생변수를 생성
- 전월의 데이터와 전년도의 데이터를 기준으로 하여 거래 내역 생성
- 수익율 계산

In [1]:
import pandas as pd 
import numpy as np 
from datetime import datetime

In [3]:
df = pd.read_csv("../../csv/MSFT.csv", index_col='Date')

In [4]:
df.head(1)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1986-03-13,0.088542,0.101563,0.088542,0.097222,0.069996,1031788800


In [5]:
# index를 시계열로 변경 
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')

In [ ]:
# index의 데이터에서 년-월 추출하여 새로운 파생변수 STD-YM에 대입

